In [12]:
import glob
import pandas as pd
import numpy as np
folder="/Users/JayLiang/Desktop/Media Storm/LIFT/programatic optimization/Till Oct 29/"
file_list=[]
for file_name in glob.glob(folder+'*.csv'):
    df = pd.read_csv(file_name,encoding="ISO-8859-1",error_bad_lines=False)
    inds = min(df['Line Item'].index[df['Line Item'].apply(lambda x: pd.isnull(x))])
    df = df.iloc[0:inds,:]
    file_list.append(df)



/Users/JayLiang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 635: expected 25 fields, saw 27\n'
b'Skipping line 151313: expected 26 fields, saw 27\n'
b'Skipping line 432: expected 26 fields, saw 27\n'
b'Skipping line 2097: expected 25 fields, saw 27\n'
b'Skipping line 283: expected 25 fields, saw 27\n'
b'Skipping line 4990: expected 26 fields, saw 27\n'
b'Skipping line 2842: expected 25 fields, saw 27\n'
b'Skipping line 1687: expected 25 fields, saw 27\n'
b'Skipping line 2212: expected 26 fields, saw 27\n'
b'Skipping line 1642: expected 25 fields, saw 27\n'
b'Skipping line 632: expected 25 fields, saw 27\n'


In [55]:
Columns=["App/URL","Browser","ISP or Carrier","Channel","Channel ID","City","City ID","Creative","Creative Width","Creative Height","Creative Type","Date",
          "Day of Week","Device Type","DMA Name","Operating System","Region","Time of Day"]
HVA=["Snap Fitness - Franchise Opportunities",
      "Snap Fitness - Franchise FAQ",
      "Snap Fitness - Franchise - What is This? Link",
      "Snap Fitness - Franchise Success Stories",
      "Snap Fitness - Franchise Request Info",
      "Snap Fitness - Franchise Thank You"]
matrix=["Clicks","Impressions","Cost","HVA","CPHVA","CPC","CTR","CPM"]
Juba_Line_Items=["Juba BT - Health & Fitness - Geo 1",
                  "Juba BT - Sports - Geo 2 (DMAs)",
                  "Juba BT - Health & Fitness - Geo 2 (DMAs)",
                  "Juba BT - Gamers - Geo 1",
                  "Juba BT - Employment - Geo 1",
                  "Juba BT - Technophiles - Geo 1",
                  "Juba BT - Movie Lover - Geo 1",
                  "Juba BT - Sports - Geo 1",
                  "Juba BT - Travel - Geo 1",
                  "Juba BT - Gamers - Geo 2 (DMAs)",
                  "Juba BT - News Junkies - Geo 2 (DMAs)",
                  "Juba BT - Movie Lover - Geo 2 (DMAs)",
                  "Juba BT - Travel - Geo 2 (DMAs)",
                  "Juba BT - Technophiles - Geo 2 (DMAs)",
                  "Juba BT - Finance - Geo 2 (DMAs)",
                  "Juba BT - Auto - Geo 2 (DMAs)",
                  "Juba BT - Employment - Geo 2 (DMAs)",
                  "Juba BT - New Mover - Geo 2 (DMAs)",
                  "Juba BT - Finance - Geo 1",
                  "Juba BT - New Mover - Geo 1",
                  "Juba BT - Auto - Geo 1",
                  "Juba BT - News Junkies - Geo 1"]
Keyword_Line_Items=["Keyword CT DESK","Keyword CT MOB"]
Retarget_Line_Items=["Site RT DESK"]
LAL_Line_Items=["LAL"]

In [172]:
for i in range(len(file_list)):
    df=file_list[i]
    df_JUBA=df[df['Line Item'].isin(Juba_Line_Items)]
    df_Keyword=df[df['Line Item'].isin(Keyword_Line_Items)]
    df_Retarget=df[df['Line Item'].isin(Retarget_Line_Items)]
    df_LAL=df[df['Line Item'].isin(LAL_Line_Items)]

    Four_Type_List=[df_JUBA,df_Keyword,df_Retarget,df_LAL] #Actually 4 type, one new "LAL" added (not yet)

    
    for j in range(len(Four_Type_List)):
        df_report_type=Four_Type_List[j]
        df_non_hva=df_report_type[[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Revenue (Adv Currency)","Total Conversions"]]
        
        if len(df_non_hva.columns)==5:
            df_merge=pd.DataFrame
            df_hva=df_hva=df_report_type[df_report_type["Conversion Pixel"] !="Total"][[col for col in df_report_type.columns if col in Columns]+["Conversion Pixel","Total Conversions"]]
            df_hva=df_hva.groupby([col for col in df_report_type.columns if col in Columns]+['Conversion Pixel'])['Total Conversions'].sum().to_frame()
            df_hva.reset_index(inplace=True)
            df_hva=df_hva.pivot(index=[col for col in df_report_type.columns if col in Columns][0],columns="Conversion Pixel",values="Total Conversions")
            df_hva.reset_index(inplace=True)
            df_hva=df_hva.fillna(0)
            
            df_non_hva=df_report_type[df_report_type["Conversion Pixel"] =="Total"][[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Revenue (Adv Currency)"]]
            df_non_hva=df_non_hva.groupby([col for col in df_report_type.columns if col in Columns])["Impressions","Clicks","Revenue (Adv Currency)"].sum()
            df_non_hva.reset_index(inplace=True)

            df_merge=pd.merge(df_non_hva,df_hva,on=[col for col in df_report_type.columns if col in Columns],how="outer")
    
        '''
        if (ncol(df_non_hva)==6){
            df_merge=NULL
            df_hva=df_report_type[which(df_report_type$Conversion.Pixel !="Total"),c(names(df_report_type)[which(names(df_report_type)%in%Columns)],"Conversion.Pixel","Total.Conversions")]
            df_hva=aggregate(df_hva$Total.Conversions,by=list(df_hva[,names(df_report_type)[which(names(df_report_type)%in%Columns)][1]],df_hva[,names(df_report_type)[which(names(df_report_type)%in%Columns)][2]],df_hva$Conversion.Pixel),sum,na.rm=TRUE)
            names(df_hva)=c(names(df_report_type)[which(names(df_report_type)%in%Columns)][1],names(df_report_type)[which(names(df_report_type)%in%Columns)][2],"Conversion.Pixel",'Total.Conversions')
            df_hva=reshape(df_hva,idvar = c(names(df_report_type)[which(names(df_report_type)%in%Columns)][1],names(df_report_type)[which(names(df_report_type)%in%Columns)][2]), timevar = "Conversion.Pixel",direction="wide")
            df_hva[is.na(df_hva)]=0
            
            df_non_hva=df_report_type[which(df_report_type$Conversion.Pixel == "Total"),c(names(df_report_type)[which(names(df_report_type)%in%Columns)],"Impressions","Clicks","Revenue..Adv.Currency.")]
            df_non_hva=aggregate.data.frame(df_non_hva[,c("Impressions","Clicks","Revenue..Adv.Currency.")],by=list(df_non_hva[,names(df_non_hva)[which(names(df_non_hva)%in%Columns)][1]],df_non_hva[,names(df_non_hva)[which(names(df_non_hva)%in%Columns)][2]]),sum,na.rm=TRUE)
            names(df_non_hva)=c(names(df_report_type)[which(names(df_report_type)%in%Columns)][1],names(df_report_type)[which(names(df_report_type)%in%Columns)][2],"Impressions","Clicks","Cost")
            
            df_merge=merge(df_non_hva,df_hva,by=c(names(df_report_type)[which(names(df_report_type)%in%Columns)][1],names(df_report_type)[which(names(df_report_type)%in%Columns)][2]),all.x=TRUE,all.y=TRUE)
        }
        '''
        df_merge.rename(columns={'Revenue (Adv Currency)':'Cost'}, inplace=True)
        df_merge['HVA']=df_merge[[col for col in df_merge.columns if col in HVA]].sum(axis=1)
        df_merge['CPHVA']=df_merge['Cost']/df_merge['HVA']
        df_merge['CPC']=df_merge['Cost']/df_merge['Clicks']
        df_merge['CTR']=df_merge['Clicks']/df_merge['Impressions']
        df_merge['CPM']=df_merge['Cost']/df_merge['Impressions']*1000
        # df_merge=df_merge[,c(names(df_merge)[which(names(df_merge) %ni% matrix & names(df_merge) %ni% Total_HVA_Reshaped)],matrix, names(df_merge)[which(names(df_merge) %in% Total_HVA_Reshaped)])]
        # names(df_merge)=c(names(df_merge)[1:which(names(df_merge)=="CPM")],sapply(names(df_merge)[(which(names(df_merge)=="CPM")+1):ncol(df_merge)],function(x) substr(x,(nchar("Total.Conversions.")+1),nchar(x))))
        # df_merge=df_merge[order(df_merge$CPHVA,df_merge$CTR,df_merge$CPC,df_merge$CPM,decreasing = c(FALSE,TRUE,FALSE,FALSE)),]
        # df_merge$Thank.You.Page<-rep(NA,nrow(df_merge))
        # writer[[j]][[i]]=df_merge
    

In [164]:
df_merge=pd.merge(df_non_hva,df_hva,on=[col for col in df_report_type.columns if col in Columns],how="outer")

In [166]:
df_merge.shape

(16580, 9)

In [167]:
df_non_hva.shape

(16580, 4)

In [168]:
df_hva.shape

(201, 6)

In [137]:
test

Conversion Pixel,Snap Fitness - Franchise - What is This? Link,Snap Fitness - Franchise FAQ,Snap Fitness - Franchise Opportunities,Snap Fitness - Franchise Success Stories,Snap Fitness - Franchise Thank You
App/URL,,,,,
12tomatoes.com,1.0,NaN,1.0,NaN,NaN
Low volume inventory,265.0,4.0,305.0,17.0,NaN
abc13.com,1.0,NaN,1.0,NaN,NaN
abc7chicago.com,NaN,NaN,NaN,1.0,NaN
actitudfem.com,2.0,NaN,2.0,NaN,NaN
activebeat.com,2.0,NaN,2.0,NaN,NaN
alloy.com,1.0,NaN,1.0,NaN,NaN
answers.com,1.0,NaN,2.0,NaN,NaN
answers.yahoo.com,NaN,NaN,1.0,1.0,NaN


In [57]:
df_report_type.columns[df_report_type.columns.isin(Columns)]

Index(['App/URL'], dtype='object')

In [53]:
Columns

['App/URL',
 'Browser',
 'ISP.or.Carrier',
 'Channel',
 'Channel.ID',
 'City',
 'City.ID',
 'Creative',
 'Creative.Width',
 'Creative.Height',
 'Creative.Type',
 'Date',
 'Day.of.Week',
 'Device.Type',
 'DMA.Name',
 'Operating.System',
 'Region',
 'Time.of.Day']

In [100]:
df_report_type.columns



Index(['Advertiser', 'Advertiser ID', 'Advertiser Status',
       'Advertiser Integration Code', 'Line Item', 'Line Item ID',
       'Line Item Status', 'Line Item Integration Code',
       'Targeted Data Providers', 'App/URL', 'App/URL ID', 'App/URL Excluded',
       'Conversion Pixel', 'Conversion Pixel ID', 'Conversion Pixel DCM ID',
       'Conversion Pixel Status', 'Conversion Pixel Integration Code',
       'Advertiser Currency', 'Impressions', 'Billable Impressions', 'Clicks',
       'Total Conversions', 'Post-Click Conversions', 'Post-View Conversions',
       'Revenue (Adv Currency)', 'Media Cost (Advertiser Currency)',
       'Click Rate (CTR)'],
      dtype='object')

In [98]:
df_non_hva=df_report_type[[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Revenue (Adv Currency)","Total Conversions"]]
        

KeyError: "['Revenue..Adv.Currency.' 'Total.Conversions'] not in index"

In [89]:
df_report_type.columns

Index(['Advertiser', 'Advertiser ID', 'Advertiser Status',
       'Advertiser Integration Code', 'Line Item', 'Line Item ID',
       'Line Item Status', 'Line Item Integration Code',
       'Targeted Data Providers', 'App/URL', 'App/URL ID', 'App/URL Excluded',
       'Conversion Pixel', 'Conversion Pixel ID', 'Conversion Pixel DCM ID',
       'Conversion Pixel Status', 'Conversion Pixel Integration Code',
       'Advertiser Currency', 'Impressions', 'Billable Impressions', 'Clicks',
       'Total Conversions', 'Post-Click Conversions', 'Post-View Conversions',
       'Revenue (Adv Currency)', 'Media Cost (Advertiser Currency)',
       'Click Rate (CTR)'],
      dtype='object')

In [93]:
test=df_report_type[[[col for col in df_report_type.columns if col in Columns],"Impressions"]]
test

TypeError: unhashable type: 'list'

In [105]:
x=pd.DataFrame

In [106]:
x

pandas.core.frame.DataFrame